In [2]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', 50)

RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)

In [3]:
# NÚMERO DE TRANSAÇÕES
N = 20000

#TAXA DE FRAUDE DESEJADA (CLASSE DESBALANCEADA)
TARGET_FRAUD_RATE = 0.03
# Em antifraude, fraude é rara (1% a 5% é comum em muitas bases)

# Gerar variáveis (features) com sentido de negócio

In [25]:
# 1) Hora da transação: 0 a 23 (inteiro)
transaction_hour = np.random.randint(0, 24, size=N)

# 2) Idade da conta em dias: contas novas tendem a ser mais arriscadas em alguns cenários
# Simular idades entre 1 e 2000 dias, mas com mais contas "mais novas" (distribuição exponencial)
account_age_days = np.clip(np.random.exponential(scale=300, size=N), 1, 2000).astype(int)

# 3) Quantidade de transações nas últimas 24h: pode indicar comportamento anormal
# Usar uma Poisson (contagem), com média maior para contas mais antigas (mais uso normal)
base_lambda = 2 + (account_age_days / 400)  # contas mais antigas tendem a ter mais transações "normais"
transactions_last_24h = np.random.poisson(lam=base_lambda).astype(int)

# 4) Transação internacional (0/1): raro, mas relevante
is_foreign_transaction = np.random.binomial(n=1, p=0.08, size=N)  # ~8% internacionais

# 5) País de alto risco (0/1): ainda mais raro
is_high_risk_country = np.random.binomial(n=1, p=0.03, size=N)    # ~3%

# 6) Ticket médio últimos 7 dias: valores em reais (simulação)
# Simular que usuários têm um padrão de gasto "pessoal"
avg_amount_last_7d = np.clip(np.random.lognormal(mean=3.6, sigma=0.6, size=N), 10, 5000)

# 7) Valor da transação (transaction_amount)
# Em antifraude, valores muito acima do ticket médio podem indicar risco.
# Simular o valor como "ticket médio * um fator aleatório"
spike_factor = np.random.lognormal(mean=0.0, sigma=0.7, size=N)  # às vezes o usuário faz compras maiores
transaction_amount = np.clip(avg_amount_last_7d * spike_factor, 1, 20000)


# Score de risco

In [27]:
# Criar um "risco" baseado em sinais comuns:
# - conta muito nova
# - transação internacional
# - país de alto risco
# - horário incomum (madrugada)
# - valor muito acima do ticket médio
# - volume muito alto de transações nas últimas 24h

# Feature auxiliar: "valor relativo" ao ticket médio
amount_ratio = transaction_amount / (avg_amount_last_7d + 1e-6)  # +1e-6 evita divisão por zero

# Feature auxiliar: horário de risco (ex: 0-5h)
late_night = ((transaction_hour >= 0) & (transaction_hour <= 5)).astype(int)

# Score linear (quanto maior, maior risco)
# Esses pesos são "arbitrados" (simulação), mas com lógica de negócio.
risk_score = (
    1.2 * is_foreign_transaction +
    2.0 * is_high_risk_country +
    0.8 * late_night +
    1.0 * (amount_ratio > 2.5).astype(int) +
    0.6 * (transactions_last_24h > 10).astype(int) +
    1.3 * (account_age_days < 30).astype(int)
)

# Além disso, adicionei um ruído pequeno para não ficar "determinístico"
risk_score = risk_score + np.random.normal(loc=0, scale=0.3, size=N)


# Converter score em probabilidade e calibrar para ~3% de fraude

In [29]:
def sigmoid(x):
    """Função logística: transforma qualquer número em probabilidade (0 a 1)."""
    return 1 / (1 + np.exp(-x))

# Achar um "intercepto" (bias) que faça a taxa média ficar próxima do TARGET_FRAUD_RATE
# Estratégia simples: testar vários valores e escolher o melhor
candidate_biases = np.linspace(-8, 0, 200)  # faixa típica de bias (negativo para deixar fraude rara)
best_bias = None
best_diff = 1e9

for b in candidate_biases:
    p = sigmoid(risk_score + b)
    diff = abs(p.mean() - TARGET_FRAUD_RATE)
    if diff < best_diff:
        best_diff = diff
        best_bias = b

# Probabilidade final
fraud_proba = sigmoid(risk_score + best_bias)

# Gerar o alvo (fraud) com base nessas probabilidades (0/1)
fraud = np.random.binomial(n=1, p=fraud_proba, size=N)

best_bias, fraud.mean()


(-4.4623115577889445, 0.03175)

In [31]:
df = pd.DataFrame({
    "transaction_amount": transaction_amount,
    "transaction_hour": transaction_hour,
    "is_foreign_transaction": is_foreign_transaction,
    "is_high_risk_country": is_high_risk_country,
    "account_age_days": account_age_days,
    "transactions_last_24h": transactions_last_24h,
    "avg_amount_last_7d": avg_amount_last_7d,
    "amount_ratio": amount_ratio,   # feature auxiliar (pode virar parte do modelo)
    "late_night": late_night,       # feature auxiliar
    "fraud": fraud
})

# Visualizar amostra
df.head()


,transaction_amount,transaction_hour,is_foreign_transaction,is_high_risk_country,account_age_days,transactions_last_24h,avg_amount_last_7d,amount_ratio,late_night,fraud
0,47.296134,21,0,0,240,4,128.577709,0.367841,0,0
1,15.669681,3,1,0,728,4,53.607539,0.292304,1,1
2,31.853713,11,0,0,290,5,23.063809,1.381112,0,0
3,70.122954,0,0,0,4,3,77.071236,0.909846,1,0
4,498.532469,16,0,0,35,3,105.107518,4.743071,0,0


In [33]:
# Taxa de fraude
print("Fraud rate:", df["fraud"].mean())

# Estatísticas rápidas
display(df.describe())

# Checar como fraude se comporta em alguns sinais
print("\nFraud rate em transações internacionais:", df.loc[df["is_foreign_transaction"] == 1, "fraud"].mean())
print("Fraud rate em país alto risco:", df.loc[df["is_high_risk_country"] == 1, "fraud"].mean())
print("Fraud rate na madrugada:", df.loc[df["late_night"] == 1, "fraud"].mean())
print("Fraud rate em contas < 30 dias:", df.loc[df["account_age_days"] < 30, "fraud"].mean())
print("Fraud rate quando amount_ratio > 2.5:", df.loc[df["amount_ratio"] > 2.5, "fraud"].mean())


Fraud rate: 0.03175


,transaction_amount,transaction_hour,is_foreign_transaction,is_high_risk_country,account_age_days,transactions_last_24h,avg_amount_last_7d,amount_ratio,late_night,fraud
count,20000.000000,20000.000000,20000.00000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000,20000.000000
mean,56.403227,11.503450,0.07830,0.031200,298.873650,2.762150,44.107227,1.277014,0.250250,0.031750
std,65.447740,6.928967,0.26865,0.173862,298.791518,1.816905,29.065330,1.023678,0.433168,0.175338
min,1.180444,0.000000,0.00000,0.000000,1.000000,0.000000,10.000000,0.064668,0.000000,0.000000
25%,19.627837,5.000000,0.00000,0.000000,86.750000,1.000000,24.588924,0.622046,0.000000,0.000000
50%,36.590692,12.000000,0.00000,0.000000,207.000000,3.000000,36.906483,1.004951,0.000000,0.000000
75%,68.045096,18.000000,0.00000,0.000000,413.000000,4.000000,55.410585,1.599618,1.000000,0.000000
max,1703.722262,23.000000,1.00000,1.000000,2000.000000,14.000000,419.668020,23.777713,1.000000,1.000000



Fraud rate em transações internacionais: 0.08876117496807152
Fraud rate em país alto risco: 0.15865384615384615
Fraud rate na madrugada: 0.05254745254745255
Fraud rate em contas < 30 dias: 0.0858768753233316
Fraud rate quando amount_ratio > 2.5: 0.06981627296587926
